**Ejercicio 1: detección de malware**

In [29]:
import numpy as np 
import pandas as pd 
import pickle 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report 

In [30]:
data = pd.read_csv('C:\\Users\\Tacos\\OneDrive\\Documentos\\Universidad\\Curso - Datos, ciberseguridad e inteligencia artificial\\Actividades\\15 dataset_train.csv') 

print(data.info()) 
print("\n --------------------------------------------- \n")
print(data.head(10))  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19611 entries, 0 to 19610
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Name                          19611 non-null  object 
 1   e_magic                       19611 non-null  int64  
 2   e_cblp                        19611 non-null  int64  
 3   e_cp                          19611 non-null  int64  
 4   e_crlc                        19611 non-null  int64  
 5   e_cparhdr                     19611 non-null  int64  
 6   e_minalloc                    19611 non-null  int64  
 7   e_maxalloc                    19611 non-null  int64  
 8   e_ss                          19611 non-null  int64  
 9   e_sp                          19611 non-null  int64  
 10  e_csum                        19611 non-null  int64  
 11  e_ip                          19611 non-null  int64  
 12  e_cs                          19611 non-null  int64  
 13  e

In [31]:
df = pd.DataFrame(data) 
X = df.drop(['Name', 'Machine', 'TimeDateStamp', 'Malware'], axis=1) 
y = df['Malware']

In [32]:
#SPLIT de 80:20
#Para garantizar la reproducibilidad de los resultados del modelo, el parámetro random_state se establece en 42

X_train, X_test, y_train, y_test = train_test_split( 
X, y, test_size=0.2, random_state=42, stratify=y 
) 
print(f'Number of features used from the dataset is {X_train.shape[1]}') 

Number of features used from the dataset is 75


In [33]:
rfc = RandomForestClassifier( 
n_estimators=100, random_state=0, oob_score=True, max_depth=16 
) 
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=16, oob_score=True, random_state=0)

In [34]:
#generación de predicciones basadas en el conjunto de prueba
y_pred = rfc.predict(X_test) 

#Informe de clasificación
print(classification_report(y_test, y_pred, 
target_names=['Benign', 'Malware'])) 

              precision    recall  f1-score   support

      Benign       0.99      0.96      0.98      1003
     Malware       0.99      1.00      0.99      2920

    accuracy                           0.99      3923
   macro avg       0.99      0.98      0.98      3923
weighted avg       0.99      0.99      0.99      3923



In [35]:
test_data = pd.read_csv("C:\\Users\\Tacos\\OneDrive\\Documentos\\Universidad\\Curso - Datos, ciberseguridad e inteligencia artificial\\Actividades\\15 dataset_test.csv") 

#Realizamos predicciones
X_pred = test_data.drop(['Name', 'Machine', 'TimeDateStamp'], axis=1) 
predictions = rfc.predict(X_pred) 

In [36]:
results = pd.DataFrame( 
{'File Name': test_data['Name'], 'Model Prediction': predictions} 
) 
print(results) 

pkl_filename = "RF_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(rfc, file) 

                                            File Name  Model Prediction
0                                  Skype-8.10.0.9.exe                 0
1                                 vlc-3.0.2-win64.exe                 0
2                                       stinger32.exe                 1
3                                SpotifyFullSetup.exe                 0
4                                    uftp_english.exe                 1
5   161a59f2525518f799c63f916c80fe85f50c5b09c74dc2...                 1
6   eaa478e65696ad5cbdb42c1b4bd6954f2a876fdde2e519...                 1
7                                   reverse_shell.exe                 1
8   873b9eaef6ea5ed6126086594529a3395bdbc5d63c97d8...                 1
9                             ScratchInstaller1.4.exe                 1
10  69eb27dd3bbf5077dcd795872535b89af9a898254b90ad...                 1
11  3334686141a400bb522824fa6f7faf30614372fe11837a...                 1
12  3ec4cb928846f8298e5a13b3e96bfc2a709cb3b005a31e...           

**Ejercicio 2: detección de intrusiones en la red**

In [37]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report  
import pickle

In [38]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent' 
    ,'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root' 
    ,'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login' 
    ,'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate' 
    ,'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count' 
    ,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate' 
    ,'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate' 
    ,'dst_host_srv_rerror_rate','label','difficulty_level']) 

train_data = pd.read_csv('C:\\Users\\Tacos\\OneDrive\\Documentos\\Universidad\\Curso - Datos, ciberseguridad e inteligencia artificial\\Actividades\\15 KDDTrain+.txt', header=None, names=columns) 
train_data.drop(['difficulty_level','num_outbound_cmds'], axis=1, inplace=True) 
le = LabelEncoder() 

train_data['protocol_type'] = le.fit_transform(train_data['protocol_type']) 
train_data['service'] = le.fit_transform(train_data['service']) 
train_data['flag'] = le.fit_transform(train_data['flag'])  
train_data['label'] = train_data['label'].apply(lambda x: 'normal' if x == 'normal' else 
'abnormal') 
print(train_data['label'].value_counts()) 

train_data['label'] = train_data['label'].apply(lambda x: 0 if x == 'normal' else 1)

label
normal      67343
abnormal    58630
Name: count, dtype: int64


In [39]:
#El conjunto de datos se divide en características (X) y etiquetas (Y)
X = train_data.drop(['label'], axis=1) 
y = train_data['label'] 

#Hacemos un SPLIT de 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40) 

In [40]:
#Estandarizaremos las características utilizando la función StandardScaler para el escalado
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

**Entrenamiento**

Entrenaremos el modelo utilizando SVM, inicializando un modelo SVM con un kernel de función de base radial (RBF, Radial basis function). El modelo se entrenará con los datos de entrenamiento estandarizados.

In [41]:
svm_model = SVC(kernel='rbf') 
svm_model.fit(X_train, y_train)

SVC()

**Predicciones**

In [42]:
y_pred = svm_model.predict(X_test)

In [43]:
print('Training accuracy: ', svm_model.score(X_train, y_train)) 
print('Testing accuracy: ', svm_model.score(X_test, y_test)) 
print('SVM Model accuracy: ', np.round(accuracy_score(y_test,y_pred),6)) 
print('Classification Report:\n', classification_report(y_test,y_pred,target_names=['Normal','Abnormal']))

Training accuracy:  0.9922092060647986
Testing accuracy:  0.9916119813717189
SVM Model accuracy:  0.991612
Classification Report:
               precision    recall  f1-score   support

      Normal       0.99      0.99      0.99     20067
    Abnormal       0.99      0.99      0.99     17725

    accuracy                           0.99     37792
   macro avg       0.99      0.99      0.99     37792
weighted avg       0.99      0.99      0.99     37792



In [45]:
#Guardar el modelo
pkl_filename = "SVM_model.pkl"  
with open(pkl_filename, 'wb') as file: 
    pickle.dump(svm_model, file) 

In [47]:
print('Starting Testing Process using the Test Dataset....') 

Starting Testing Process using the Test Dataset....


In [49]:
test_data = pd.read_csv('C:\\Users\\Tacos\\OneDrive\\Documentos\\Universidad\\Curso - Datos, ciberseguridad e inteligencia artificial\\Actividades\\15 KDDTest+.txt', header=None, names=columns) 
test_data.drop(['difficulty_level', 'num_outbound_cmds'], axis=1, inplace=True) 

In [50]:
le = LabelEncoder() 
test_data['protocol_type'] = le.fit_transform(test_data['protocol_type']) 
test_data['service'] = le.fit_transform(test_data['service']) 
test_data['flag'] = le.fit_transform(test_data['flag']) 
test_data['label'] = test_data['label'].apply(lambda x: 0 if x == 'normal' else 1)

In [51]:
X_test = test_data.drop(['label'], axis=1) 
y_test = test_data['label']

X_test = scaler.transform(X_test) 
y_pred = svm_model.predict(X_test)

In [52]:
print('Testing Phase accuracy: ', accuracy_score(y_test, y_pred)) 
print('SVM Model accuracy: ', accuracy_score(y_test, y_pred)) 
print('Classification Report:\n', classification_report(y_test, y_pred, target_names=["Normal", "Abnormal"]))

Testing Phase accuracy:  0.7819819020581973
SVM Model accuracy:  0.7819819020581973
Classification Report:
               precision    recall  f1-score   support

      Normal       0.67      0.98      0.79      9711
    Abnormal       0.98      0.63      0.77     12833

    accuracy                           0.78     22544
   macro avg       0.82      0.81      0.78     22544
weighted avg       0.84      0.78      0.78     22544



Usamos un conjunto de entrenamiento, uno de pruebas y otro de validación (similar a unas propuestas vistas en mpaquinas de aprendizaje).

La falta de selección de características puede contribuir al sobreajuste del modelo de SVM (esto se observa en la desviación obtenida). El uso de un kernel RBF, susceptible al sobreajuste en conjuntos de datos pequeños, agrava aún más este problema. 

**Pasando de la detección a la clasificación**

La transición de la detección de malware a la clasificación de malware representa una 
evolución significativa en la sofisticación y la granularidad del análisis realizado sobre 
software potencialmente dañino.

En el ámbito de la detección de malware, el objetivo principal es identificar si un determinado software exhibe un comportamiento malicioso o no.  Los algoritmos utilizados para la detección de malware se centran en distinguir entre estas dos clases, a menudo empleando técnicas como la detección de anomalías o el reconocimiento de patrones para señalar actividad sospechosa. Por otro lado, la clasificación de malware profundiza en la categorización y caracterización del software malicioso, con el objetivo de clasificar el malware en diferentes tipos o familias en función de sus patrones de comportamiento, estructuras de código u otros atributos.